In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.elasticsearch:elasticsearch-spark-30_2.12:8.11.0 pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_timestamp, when, window, expr
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType

# Créer une session Spark
spark = SparkSession.builder \
    .appName("IoT Kafka") \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
            "org.elasticsearch:elasticsearch-spark-30_2.12:8.12.2") \
    .getOrCreate()

# Schéma des données JSON
schema = StructType() \
    .add("device_id", StringType()) \
    .add("temperature", DoubleType()) \
    .add("humidity", DoubleType()) \
    .add("pressure", DoubleType()) \
    .add("vibration", DoubleType()) \
    .add("latitude", DoubleType()) \
    .add("longitude", DoubleType()) \
    .add("timestamp", StringType())

# Lire depuis Kafka
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:9092") \
    .option("subscribe", "iot_raw_data") \
    .option("startingOffsets", "latest") \
    .load()

# Extraire le JSON
parsed_df = raw_df.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# Conversion du timestamp
parsed_df = parsed_df.withColumn("timestamp", to_timestamp("timestamp"))

# Nettoyage des valeurs manquantes
clean_df = parsed_df.dropna()

# Ajout des règles métiers
alert_df = clean_df.withColumn(
    "status",
    when(col("temperature") > 50, "Alerte")
    .when((col("humidity") < 10) & (col("temperature") > 40), "Critique")
    .when((col("vibration") > 5) & ((col("pressure") < 950) | (col("pressure") > 1050)), "Alerte")
    .otherwise("Normal")
)

# Capteurs inactifs (si aucune donnée depuis >1h) => traitement par une autre logique (batch/join avec état)

# Exemple de corrélation régionale : moyenne température par région (arrondi lat/lon à 0.1°)
regional_alerts = alert_df \
    .withColumn("region_lat", expr("round(latitude, 1)")) \
    .withColumn("region_lon", expr("round(longitude, 1)")) \
    .groupBy(window("timestamp", "10 minutes"), "region_lat", "region_lon") \
    .avg("temperature", "humidity") \
    .withColumnRenamed("avg(temperature)", "avg_temp") \
    .withColumnRenamed("avg(humidity)", "avg_humidity") \
    .withColumn("regional_status", when(col("avg_temp") > 45, "Alerte regionale").otherwise("OK"))

# Écriture dans Elasticsearch
query = alert_df.writeStream \
    .format("org.elasticsearch.spark.sql") \
    .option("checkpointLocation", "/tmp/checkpoint_iot") \
    .option("es.nodes", "es01") \
    .option("es.port", "9200") \
    .option("es.nodes.wan.only", "true") \
    .option("es.net.ssl", "true") \
    .option("es.net.ssl.cert.allow.self.signed", "true") \
    .option("es.net.http.auth.user", "elastic") \
    .option("es.net.http.auth.pass", "Eselpil2") \
    .option("es.resource", "iot_data_enriched") \
    .option("es.mapping.id", "unique_id") \
    .outputMode("update") \
    .start()


query.awaitTermination()

In [ ]:
!apt update && apt install -y netcat
!nc -zv broker 9092



In [1]:
import requests
requests.get("https://es01:9200/", auth=("elastic", "Eselpil2"), verify=False)



/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [5]:
!curl -u elastic:Eselpil2 https://es01:9200 -k


{
  "name" : "es01",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "q_ulNIZXSFC5tyGRle5htQ",
  "version" : {
    "number" : "8.15.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "253e8544a65ad44581194068936f2a5d57c2c051",
    "build_date" : "2024-09-02T22:04:47.310170297Z",
    "build_snapshot" : false,
    "lucene_version" : "9.11.1",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import pyspark
print(pyspark.__version__)


3.5.0


In [2]:
!pip install elasticsearch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.8/895.8 kB 3.3 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 16.6 MB/s eta 0:00:00


In [3]:
import os
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_timestamp, when, expr
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType

# Configurer Spark avec les bons packages
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.elasticsearch:elasticsearch-spark-30_2.12:8.11.0 pyspark-shell'

# Créer une session Spark
spark = SparkSession.builder \
    .appName("IoT Kafka") \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
            "org.elasticsearch:elasticsearch-spark-30_2.12:8.12.2") \
    .getOrCreate()

# Schéma des données JSON
schema = StructType() \
    .add("device_id", StringType()) \
    .add("temperature", DoubleType()) \
    .add("humidity", DoubleType()) \
    .add("pressure", DoubleType()) \
    .add("vibration", DoubleType()) \
    .add("latitude", DoubleType()) \
    .add("longitude", DoubleType()) \
    .add("timestamp", StringType())

# Lire depuis Kafka
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:9092") \
    .option("subscribe", "iot_raw_data") \
    .option("startingOffsets", "latest") \
    .load()

# Extraire le JSON
parsed_df = raw_df.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# Conversion du timestamp
parsed_df = parsed_df.withColumn("timestamp", to_timestamp("timestamp"))

# Nettoyage des valeurs manquantes
clean_df = parsed_df.dropna()

# Ajout des règles métiers
alert_df = clean_df.withColumn(
    "status",
    when(col("temperature") > 50, "Alerte")
    .when((col("humidity") < 10) & (col("temperature") > 40), "Critique")
    .when((col("vibration") > 5) & ((col("pressure") < 950) | (col("pressure") > 1050)), "Alerte")
    .otherwise("Normal")
)

# Exemple de corrélation régionale : moyenne température par région (arrondi lat/lon à 0.1°)
regional_alerts = alert_df \
    .withColumn("region_lat", expr("round(latitude, 1)")) \
    .withColumn("region_lon", expr("round(longitude, 1)")) \
    .groupBy("region_lat", "region_lon", "timestamp") \
    .avg("temperature", "humidity") \
    .withColumnRenamed("avg(temperature)", "avg_temp") \
    .withColumnRenamed("avg(humidity)", "avg_humidity") \
    .withColumn("regional_status", when(col("avg_temp") > 45, "Alerte régionale").otherwise("OK"))

# Fonction pour écrire dans Elasticsearch à chaque micro-batch
def write_to_es(rdd, batch_id):
    if not rdd.isEmpty():
        es_write_conf = {
            "es.nodes": "es01",
            "es.port": "9200",
            "es.resource": "iot_data_enriched/doc",
            "es.input.json": "yes",
            "es.mapping.id": "unique_id",
            "es.net.ssl": "true",
            "es.net.ssl.cert.allow.self.signed": "true",
            "es.nodes.wan.only": "true",
            "es.net.http.auth.user": "elastic",
            "es.net.http.auth.pass": "Eselpil2"
        }

        # Envoi des données à Elasticsearch
        rdd.map(lambda row: (None, json.dumps(row.asDict()))) \
            .saveAsNewAPIHadoopFile(
                path='-',
                outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
                keyClass="org.apache.hadoop.io.NullWritable",
                valueClass="org.apache.hadoop.io.Text",
                conf=es_write_conf
            )

# Écriture de chaque micro-batch dans Elasticsearch via foreachRDD
query = regional_alerts.writeStream \
    .foreachBatch(write_to_es) \
    .outputMode("update") \
    .option("checkpointLocation", "/tmp/checkpoint_iot") \
    .start()

query.awaitTermination()


ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_2898/4093405898.py", line 69, in write_to_es
    if not rdd.isEmpty():
           ^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/dataframe.py", line 883, in isEmpty
    return self._jdf.isEmpty()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
            

StreamingQueryException: [STREAM_FAILED] Query [id = 789b5a84-8f7a-4a24-842f-e1c828017d67, runId = 707d81e2-36d4-44be-94ef-5d3393e9b2c3] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_2898/4093405898.py", line 69, in write_to_es
    if not rdd.isEmpty():
           ^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/sql/dataframe.py", line 883, in isEmpty
    return self._jdf.isEmpty()
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/pyt